In [161]:
import csv
import numpy as np
import scipy.special as sps
import math
import random
import gmpy2
import mpmath
lambdaij=1

In [162]:
PATH = "/Users/siyu/Downloads/stanford-aa228-master/"
FileNames = ["titanic.csv", "whitewine.csv", "schoolgrades.csv"]

In [163]:
def read_csv(PATH, FileNames):
    with open(PATH+FileNames, 'r') as f:
        reader = csv.reader(f)
        data = list(reader)
    return data
#len(data)

In [164]:
def init_data_structures(data):
    feature_dim = len(data[0]);
    ordered_list = []
    for i in range(0, feature_dim):
        ordered_list.append(i)
    print(ordered_list)
    instances = []
    inst_counts = []
    npdata1 = np.array(data[0:len(data)])
    print(npdata1)
    np.random.shuffle(np.transpose(npdata1))
    print(npdata1)
    npdata = npdata1[1:len(npdata1)]
    print(npdata)
    for i in ordered_list:
        unique, counts = np.unique(npdata[:,i], return_counts=True)
        instances.append(unique)
        inst_counts.append(counts)
    print(instances)
    print(inst_counts)
    return ordered_list, npdata, instances, inst_counts

In [165]:
def init_dag(ordered_list):
    dag = []
    for i in ordered_list:
        dag.append([])    
    return dag

In [166]:
def find_p_value_list(instances, parents, j):
    value_list = []
    i = 0
    if j == 0:
        for i in range(0, len(parents)):
            value_list.append(instances[parents[i]][0]) 
    else:
        for i in range(0, len(parents)):
            idx = 0 if j==0 else j%len(instances[parents[i]])
            #print(idx)
            value_list.append(instances[parents[i]][idx])
            j = (int)(j/len(instances[parents[i]]))
    return value_list

In [167]:
def find_m_ij0(instances, parents, j, npdata):
    value_list = find_p_value_list(instances, parents, j)
    m_ij0 = 0
    for row in npdata:
        noupdate = False
        for idx in range(0, len(parents)):
            if row[parents[idx]] != value_list[idx]:
                noupdate = True
                break
        if not noupdate:
            m_ij0 += 1
    return m_ij0
#find_m_ij0([0,1,2], 5, npdata)

In [168]:
def find_m_ijk(instances, node, parents, j, kth, npdata):
    value_list = find_p_value_list(instances, parents, j)
    m_ijk = 0
    for row in npdata:
        noupdate = False
        if row[node] != instances[node][kth]:
            continue
        for idx in range(0, len(parents)):
            if row[parents[idx]] != value_list[idx]:
                noupdate = True
                break;
        if not noupdate:
            m_ijk += 1
    return m_ijk
#find_m_ijk(3,[0,1,2],5, 1, npdata)

In [169]:
#mp.dps=300
#from gmpy2 import mpfr
from mpmath import *
def bayesian_score(ordered_list, dag, npdata, instances, inst_counts):
    f1 = mpf(0)
    for node in ordered_list:
        parents = dag[node]
        pinstances = 1
        for parent in parents:
            pinstances *= len(instances[parent])
        for j in range(0, pinstances):
            m_ij0 = find_m_ij0(instances, parents, j, npdata)
            alpha_ij0 = len(instances[node])*lambdaij
            f1 += mpmath.log(mpmath.gamma(alpha_ij0)/mpmath.gamma(alpha_ij0+m_ij0))
            f2 = mpf(0)
            for kth in range(0, len(instances[node])):
                m_ijk = find_m_ijk(instances, node, parents, j, kth, npdata)
                alpha_ijk = mpf(1)
                f2 += mpmath.log(mpmath.gamma(alpha_ijk+m_ijk)/mpmath.gamma(alpha_ijk))
            f1 += f2
    return f1
#dag = init_dag()
#dag[1].append(3)
#bayesian_score(dag, npdata)

In [170]:
def dfs(node, dag, permmark, tempmark):
    if tempmark[node]==1:
        return False
    if permmark[node]==1:
        return True
    tempmark[node] = 1
    for parent in dag[node]:
        if not dfs(parent, dag, permmark, tempmark):
            return False
    permmark[node] = 1
    tempmark[node] = 0
    return True

In [171]:
def isDAG(dag):
    permmark = []
    tempmark = []
    for i in range(0, len(dag)):
        permmark.append(0)
        tempmark.append(0)
    for node in range(0, len(dag)):
        if permmark[node] == 0:
            if not dfs(node, dag, permmark, tempmark):
                return False
    return True
#isDAG([[1,2],[1],[],[0]])

In [172]:
from mpmath import *
def k2search(ordered_list, npdata, instances, inst_counts, max_p):
    dag = init_dag(ordered_list)
    print(dag)
    max_score = mpf(-inf)
    for node in ordered_list:
        while len(dag[node]) < max_p:
            max_node = -1
            for parent in ordered_list:
                if node==parent:
                    continue
                if parent in dag[node]:
                    continue
                dag[node].append(parent)
                if not isDAG(dag):
                    del dag[node][-1]
                    continue
                s1=bayesian_score(ordered_list, dag, npdata, instances, inst_counts)
                if s1 > max_score:
                    max_score = s1
                    max_node = parent
                del dag[node][-1]
            #assert max_node != -1
            if max_node == -1:
                break
            print("Iterations:(node_idx, max_scored parent, max_score)", node, max_node, max_score)
            dag[node].append(max_node)
    print ("Final:", max_score)
    return dag         
#k2search(ordered_list, npdata, 2)

In [173]:
print(npdata1[0])

['fare' 'numparentschildren' 'survived' 'sex' 'portembarked' 'age'
 'passengerclass' 'numsiblings']


In [174]:
def main():
    for file in FileNames:
        data = read_csv(PATH, file)
        print(len(data))
        ordered_list, npdata, instances, inst_counts = init_data_structures(data)
        dag = k2search(ordered_list, npdata, instances, inst_counts, 2)
        print(dag)

In [175]:
main()

890
[0, 1, 2, 3, 4, 5, 6, 7]
[['age' 'portembarked' 'fare' ..., 'sex' 'numsiblings' 'survived']
 ['1' '1' '1' ..., '1' '1' '1']
 ['2' '2' '1' ..., '2' '1' '2']
 ..., 
 ['2' '1' '1' ..., '2' '1' '1']
 ['1' '2' '1' ..., '1' '1' '2']
 ['2' '3' '1' ..., '1' '1' '1']]
[['numparentschildren' 'sex' 'passengerclass' ..., 'age' 'portembarked'
  'numsiblings']
 ['1' '1' '1' ..., '1' '1' '1']
 ['1' '2' '2' ..., '2' '2' '1']
 ..., 
 ['2' '2' '1' ..., '2' '1' '1']
 ['1' '1' '2' ..., '1' '2' '1']
 ['1' '1' '1' ..., '2' '3' '1']]
[['1' '1' '1' ..., '1' '1' '1']
 ['1' '2' '2' ..., '2' '2' '1']
 ['1' '2' '1' ..., '1' '1' '1']
 ..., 
 ['2' '2' '1' ..., '2' '1' '1']
 ['1' '1' '2' ..., '1' '2' '1']
 ['1' '1' '1' ..., '2' '3' '1']]
[array(['1', '2', '3'], 
      dtype='<U18'), array(['1', '2'], 
      dtype='<U18'), array(['1', '2', '3'], 
      dtype='<U18'), array(['1', '2', '3'], 
      dtype='<U18'), array(['1', '2'], 
      dtype='<U18'), array(['1', '2', '3'], 
      dtype='<U18'), array(['1', '2', '

Iterations:(node_idx, max_scored parent, max_score) 0 17 -77030.9999091769
Iterations:(node_idx, max_scored parent, max_score) 1 17 -76424.9813720671
Iterations:(node_idx, max_scored parent, max_score) 1 3 -76380.7534003907
Iterations:(node_idx, max_scored parent, max_score) 2 5 -76219.3787235148
Iterations:(node_idx, max_scored parent, max_score) 2 25 -75974.1376147393
Iterations:(node_idx, max_scored parent, max_score) 3 27 -73323.4983423833
Iterations:(node_idx, max_scored parent, max_score) 4 2 -73189.2491795815
Iterations:(node_idx, max_scored parent, max_score) 4 14 -73104.3635271547
Iterations:(node_idx, max_scored parent, max_score) 5 6 -72870.0834475856
Iterations:(node_idx, max_scored parent, max_score) 5 26 -72266.6527770741
Iterations:(node_idx, max_scored parent, max_score) 6 20 -68869.8704818384
Iterations:(node_idx, max_scored parent, max_score) 7 6 -68274.7311154429
Iterations:(node_idx, max_scored parent, max_score) 7 24 -67968.0216667036
Iterations:(node_idx, max_scor